In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt #  plotting and data visualization
import seaborn as sns # improve visuals
sns.set() # Set as default style

import string # python library
import re # regex library

from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_punctuation, strip_multiple_whitespaces, strip_numeric, remove_stopwords, strip_short # Preprocesssing
from gensim.models import Word2Vec # Word2vec

from sklearn import cluster # Kmeans clustering
from sklearn import metrics # Metrics for evaluation
from sklearn.decomposition import PCA #PCA
from sklearn.manifold import TSNE #TSNE

In [2]:
fake = pd.read_csv('C:/Users/Admin/Desktop/Fake.csv')
true = pd.read_csv('C:/Users/Admin/Desktop/True.csv')

In [3]:
fake.head(10)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017"
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017"
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017"
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017"
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017"


In [4]:
true.head(10)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
5,"White House, Congress prepare for talks on spe...","WEST PALM BEACH, Fla./WASHINGTON (Reuters) - T...",politicsNews,"December 29, 2017"
6,"Trump says Russia probe will be fair, but time...","WEST PALM BEACH, Fla (Reuters) - President Don...",politicsNews,"December 29, 2017"
7,Factbox: Trump on Twitter (Dec 29) - Approval ...,The following statements were posted to the ve...,politicsNews,"December 29, 2017"
8,Trump on Twitter (Dec 28) - Global Warming,The following statements were posted to the ve...,politicsNews,"December 29, 2017"
9,Alabama official to certify Senator-elect Jone...,WASHINGTON (Reuters) - Alabama Secretary of St...,politicsNews,"December 28, 2017"


In [5]:
# The following is a crude way to remove the @realDonaldTrump tweet disclaimer and State/Publisher at start of text

cleansed_data = []
for data in true.text:
    if "@realDonaldTrump : - " in data:
        cleansed_data.append(data.split("@realDonaldTrump : - ")[1])
    elif "(Reuters) -" in data:
        cleansed_data.append(data.split("(Reuters) - ")[1])
    else:
        cleansed_data.append(data)

true["text"] = cleansed_data
true.head(10)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",The head of a conservative Republican faction ...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,Transgender people will be allowed for the fir...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,The special counsel investigation of links bet...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,Trump campaign adviser George Papadopoulos tol...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,President Donald Trump called on the U.S. Post...,politicsNews,"December 29, 2017"
5,"White House, Congress prepare for talks on spe...",The White House said on Friday it was set to k...,politicsNews,"December 29, 2017"
6,"Trump says Russia probe will be fair, but time...",President Donald Trump said on Thursday he bel...,politicsNews,"December 29, 2017"
7,Factbox: Trump on Twitter (Dec 29) - Approval ...,While the Fake News loves to talk about my so-...,politicsNews,"December 29, 2017"
8,Trump on Twitter (Dec 28) - Global Warming,"Together, we are MAKING AMERICA GREAT AGAIN! b...",politicsNews,"December 29, 2017"
9,Alabama official to certify Senator-elect Jone...,Alabama Secretary of State John Merrill said h...,politicsNews,"December 28, 2017"


In [7]:
true.text[7]

'While the Fake News loves to talk about my so-called low approval rating, @foxandfriends just showed that my rating on Dec. 28, 2017, was approximately the same as President Obama on Dec. 28, 2009, which was 47%...and this despite massive negative Trump coverage & Russia hoax! [0746 EST] - Why is the United States Post Office, which is losing many billions of dollars a year, while charging Amazon and others so little to deliver their packages, making Amazon richer and the Post Office dumber and poorer? Should be charging MUCH MORE! [0804 EST] -- Source link: (bit.ly/2jBh4LU) (bit.ly/2jpEXYR) '

In [8]:
# Merging title and text
fake['Sentences'] = fake['title'] + ' ' + fake['text']
true['Sentences'] = true['title'] + ' ' + true['text']

# Adding fake and true label
fake['Label'] = 0
true['Label'] = 1

# We can merge both together since we now have labels
final_data = pd.concat([fake, true])

# Randomize the rows so its all mixed up
final_data = final_data.sample(frac=1).reset_index(drop=True)

# Drop columns not needed
final_data = final_data.drop(['title', 'text', 'subject', 'date'], axis = 1)

final_data.head(10)

,Sentences,Label
0,"Trump calls for waterboarding, other methods i...",1
1,Report: Trump Talking To Fox News Birther For...,0
2,"Eyewitness Says Feds Ambushed Bundys, 100 Shot...",0
3,The Worst Possible Person Just Won Alabama’s ...,0
4,Canada spy agency settles lawsuit alleging hom...,1
5,Cruz & Kasich Quit: Trump Crushes Elite Establ...,0
6,French unions block fuel depots in protest aga...,1
7,Prominent Attorney And Law Professor Points O...,0
8,[VIDEO] FLASHBACK…MARTIN LUTHER KING JR. ON RI...,0
9,Black Politicians Increase Attacks on Ben Cars...,0


In [10]:
# Here we preprocess the sentences
def remove_URL(s):
    regex = re.compile(r'https?://\S+|www\.\S+|bit\.ly\S+')
    return regex.sub(r'',s)

# Preprocessing functions to remove lowercase, links, whitespace, tags, numbers, punctuation, strip words
CUSTOM_FILTERS = [lambda x: x.lower(), strip_tags, remove_URL, strip_punctuation, strip_multiple_whitespaces, strip_numeric, remove_stopwords, strip_short]

# Here we store the processed sentences and their label
processed_data = []
processed_labels = []

for index, row in final_data.iterrows():
    words_broken_up = preprocess_string(row['Sentences'], CUSTOM_FILTERS)
    # This eliminates any fields that may be blank after preprocessing
    if len(words_broken_up) > 0:
        processed_data.append(words_broken_up)
        processed_labels.append(row['Label'])

In [11]:
model = Word2Vec(processed_data, min_count=1)
model.wv.most_similar("country")

[('nation', 0.8168737888336182),
 ('america', 0.6745514869689941),
 ('countries', 0.5909740924835205),
 ('europe', 0.5711232423782349),
 ('world', 0.5271807909011841),
 ('prosperous', 0.5138536095619202),
 ('americans', 0.5120750665664673),
 ('societies', 0.5099357962608337),
 ('realize', 0.5072239637374878),
 ('continent', 0.49530842900276184)]

In [12]:
# Getting the vector of a sentence based on average of all the word vectors in the sentence
# We get the average as this accounts for different sentence lengths

def ReturnVector(x):
    try:
        return model[x]
    except:
        return np.zeros(100)
    
def Sentence_Vector(sentence):
    word_vectors = list(map(lambda x: ReturnVector(x), sentence))
    return np.average(word_vectors, axis=0).tolist()

X = []
for data_x in processed_data:
    X.append(Sentence_Vector(data_x))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [13]:
X_np = np.array(X)
X_np.shape

(44889, 100)

In [14]:
# Training for 2 clusters (Fake and Real)
kmeans = cluster.KMeans(n_clusters=2, verbose=1)

# Fit predict will return labels
clustered = kmeans.fit_predict(X_np)

Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 453465.3476931369
start iteration
done sorting
end inner loop
Iteration 1, inertia 447761.05468068534
start iteration
done sorting
end inner loop
Iteration 2, inertia 445982.6297782133
start iteration
done sorting
end inner loop
Iteration 3, inertia 445118.7092864347
start iteration
done sorting
end inner loop
Iteration 4, inertia 444678.0674062467
start iteration
done sorting
end inner loop
Iteration 5, inertia 444485.1738809135
start iteration
done sorting
end inner loop
Iteration 6, inertia 444415.3815645248
start iteration
done sorting
end inner loop
Iteration 7, inertia 444384.8693476604
start iteration
done sorting
end inner loop
Iteration 8, inertia 444373.9354605433
start iteration
done sorting
end inner loop
Iteration 9, inertia 444370.37268233194
start iteration
done sorting
end inner loop
Iteration 10, inertia 444368.9178690634
start iteration
done sorting
end inner loop
Iteration 11, in

Iteration 0, inertia 463891.3927424545
start iteration
done sorting
end inner loop
Iteration 1, inertia 448473.8436678703
start iteration
done sorting
end inner loop
Iteration 2, inertia 446538.7800520378
start iteration
done sorting
end inner loop
Iteration 3, inertia 445456.4617567284
start iteration
done sorting
end inner loop
Iteration 4, inertia 444834.9318138398
start iteration
done sorting
end inner loop
Iteration 5, inertia 444553.0937576164
start iteration
done sorting
end inner loop
Iteration 6, inertia 444440.7902627855
start iteration
done sorting
end inner loop
Iteration 7, inertia 444396.1426334432
start iteration
done sorting
end inner loop
Iteration 8, inertia 444378.35632443696
start iteration
done sorting
end inner loop
Iteration 9, inertia 444372.1023617089
start iteration
done sorting
end inner loop
Iteration 10, inertia 444369.6815446856
start iteration
done sorting
end inner loop
Iteration 11, inertia 444368.7106271676
start iteration
done sorting
end inner loop
I

In [15]:
testing_df = {'Sentence': processed_data, 'Labels': processed_labels, 'Prediction': clustered}
testing_df = pd.DataFrame(data=testing_df)

testing_df.head(10)

,Sentence,Labels,Prediction
0,"[trump, calls, waterboarding, methods, fight, ...",1,0
1,"[report, trump, talking, fox, news, birther, a...",0,0
2,"[eyewitness, says, feds, ambushed, bundys, sho...",0,0
3,"[worst, possible, person, won, alabama’s, runo...",0,0
4,"[canada, spy, agency, settles, lawsuit, allegi...",1,1
5,"[cruz, kasich, quit, trump, crushes, elite, es...",0,0
6,"[french, unions, block, fuel, depots, protest,...",1,1
7,"[prominent, attorney, law, professor, points, ...",0,0
8,"[video, flashback…martin, luther, king, riots,...",0,0
9,"[black, politicians, increase, attacks, ben, c...",0,0


In [16]:
correct = 0
incorrect = 0
for index, row in testing_df.iterrows():
    if row['Labels'] == row['Prediction']:
        correct += 1
    else:
        incorrect += 1
        
print("Correctly clustered news: " + str((correct*100)/(correct+incorrect)) + "%")

Correctly clustered news: 87.33097195303972%
